# Second method to filter out replicate microstates

The microscopic pKa refers to the pKa of deprotonation of a single titratable group while all the other titratable and tautomerizable functional groups of the same molecule are held fixed.

Resonance structures or geometric isomers of compounds do not constitute seperate microstates. In this notebook I compare number of heavy atoms, total charge of the molecule and number of hydrogens bound to each heavy atom to detect unique or replicate microstate structures.

In [1]:
import pandas as pd
import numpy as np
from openeye import oechem, oedepict
import oenotebook as oenb
from IPython.display import HTML, display
from base64 import b64encode

In [21]:
def get_labeled_mol(smiles, label='heavy'):
    """
    returns an OEMol with heavy atoms labeled with a specific indice
    """
    mol = oechem.OEMol()
    if not oechem.OESmilesToMol(mol, smiles):
        print("Couldn't parse smiles (%s) returning None" % smiles)
        return None
    
    for idx, a in enumerate(mol.GetAtomIter(oechem.OEIsHeavy())):
        a.SetData('heavy', idx+1)
    
    return mol

def get_total_charge(mol):
    """
    Calculates total formal charge of a molecule.
    
    Input
    mol: oechem.OEMol() object
    """
    total_charge = 0
    for a in mol.GetAtomIter():
        total_charge += a.GetFormalCharge()
    return total_charge

def compare_total_charge(mol1, mol2):
    """
    Compares total formal charge of two SMILES strings.
    Returns True of they have equal formal charge.
    """
    total_charge1 = get_total_charge(mol1)
    total_charge2 = get_total_charge(mol2)
    if total_charge1 == total_charge2:
        return True
    else:
        return False
    
def calculate_total_charge_difference(mol1, mol2):
    charge_difference = get_total_charge(mol1) - get_total_charge(mol2)
    return charge_difference

def count_heavy_atoms(mol):
    """
    Returns number of heavy atoms in a molecule.
    """
    heavy_atom_count = 0
    for idx, atom in enumerate(mol.GetAtomIter()):
        heavy_atom_count = idx + 1
        #print(atom, atom.GetData()) # Target mol has labelled heavy atoms
    return heavy_atom_count

def get_total_hydrogen_count(mol):
    """
    Calculates total number of hydrogens in a structure.
    """
    hydrogen_count = 0
    
    for idx, atom in enumerate(mol.GetAtomIter()):
        hydrogen_count += atom.GetTotalHCount()
    
    return hydrogen_count

def get_mcss(pattern, target):
    """
    Finds maximum common substructure based on atomic number criteria 
    
    Returns the match as a mol object and a dictionary that maps pattern heavy atom labels to target labels .
    
    MCSS should result in only 1 match.
    """
    atomexpr = oechem.OEExprOpts_AtomicNumber
    bondexpr = oechem.OEExprOpts_EqSingleDouble # single or double bonds are considered identical

    #bondexpr = oechem.OEExprOpts_DefaultBonds
    #atomexpr = oechem.OEExprOpts_DefaultAtoms 
    
    # create maximum common substructure object
    mcss = oechem.OEMCSSearch(pattern, atomexpr, bondexpr, oechem.OEMCSType_Exhaustive)
    
    # set scoring function
    #mcss.SetMCSFunc(oechem.OEMCSMaxAtoms())
    mcss.SetMCSFunc(oechem.OEMCSMaxAtomsCompleteCycles())

    # ignore matches smaller than 6 atoms
    mcss.SetMinAtoms(6)
    unique = True
    
    # create a dictionary that maps pattern heavy atom labels to target labels
    label_dict = {}

    # loop over matches
    for i, match in enumerate(mcss.Match(target, unique)):
        count = i + 1
        print ("Match %d:" % (count))
        
        print ("pattern atoms:", end=" ")
        for ma in match.GetAtoms():
            print (ma.pattern.GetIdx(), end=" ")
        print ("\ntarget atoms: ", end=" ")
        for ma in match.GetAtoms():
            print (ma.target.GetIdx(), end=" ")
        print()
        
        print ("\npattern heavy atom labels:", end=" ")
        for ma in match.GetAtoms():
            print (ma.pattern.GetData("heavy"), end=" ")
        print ("\ntarget heavy atom labels: ", end=" ")
        for ma in match.GetAtoms():
            print (ma.target.GetData("heavy"), end=" ")
        print()
        
        # record matching pattern and target labels in a dictionary
        for ma in match.GetAtoms():
            label_dict[ma.pattern.GetData("heavy")] = ma.target.GetData("heavy")

        # create match subgraph
        m = oechem.OEGraphMol()
        oechem.OESubsetMol(m, match, True)

        print ("\nmatch smiles =", oechem.OEMolToSmiles(m))

    # check if there is only single match
    if (count != 1):
        print("Warning! There is multiple matches.")
    else:
        print("Exactly one match.")
        
    return m, label_dict, mcss

def get_labelled_heavy_atom_Hcount(mol, heavy_atom_label, data_section = "heavy"):
    """
    finds the heavy atom with the same label in a molecule and returns its H-count
    """
    query_label = heavy_atom_label
    for i, atom in enumerate(mol.GetAtomIter()):
        pattern_label = atom.GetData(data_section)
        if pattern_label == query_label:
            Hcount = atom.GetTotalHCount()
            return Hcount
    # if not found
    return np.NaN

def check_how_many_heavy_atoms_changed_protonation_state(pattern, target, label_dict):
    """
    Returns the number of labeled heavy atoms that have different number of bound hydrogens between
    two proposed microstates of the same molecule.
    """
    
    # create numpy arrays to store heavy atom hydrogen counts
    pattern_HCount_array = np.zeros(count_heavy_atoms(pattern))
    target_HCount_array = np.zeros(count_heavy_atoms(target))
    #print("Pattern number of heavy atoms: ", count_heavy_atoms(pattern))
    #print("Target number of heavy atoms:  ", count_heavy_atoms(target))
    #print("label_dict:")
    #print(label_dict)
    #print("len pattern_HCount_array: ", len(pattern_HCount_array))
    #print("len target_HCount_array: ", len(target_HCount_array))

    # iterate over label dictionary to and populate Hcount arrays
    for ii, (key, value) in enumerate(label_dict.items()):
        #print(ii, key, value)
        p_label = key
        t_label = value

        # find the heavy atom with the same label in pattern molecule and get H-count
        Hcount = get_labelled_heavy_atom_Hcount(pattern, p_label)
        #print("ii: ", ii)
        pattern_HCount_array[ii] = Hcount

        # find the heavy atom with the same label in target molecule and get H-count
        Hcount = get_labelled_heavy_atom_Hcount(target, t_label)
        target_HCount_array[ii] = Hcount

    #print("Pattern Hcount: ", pattern_HCount_array)
    #print("Target Hcount: ", target_HCount_array)

    # calculate difference of 2 arrays
    Hcount_difference_array = np.subtract(pattern_HCount_array,target_HCount_array)
    #print("Difference in Hcount:", Hcount_difference_array )

    # calculate the number of heavy atoms protonated or deprotonated
    prot_deprot_heavy_atom_count = 0
    for diff in Hcount_difference_array:
        if diff != 0.0:
            prot_deprot_heavy_atom_count += 1

    print("Number of heavy atoms with different number of hydrogens: ", prot_deprot_heavy_atom_count)
    return prot_deprot_heavy_atom_count

def is_physical_microstate_pair(pattern, target):
    
    # Compare number of heavy atoms
    difference_of_heavy_atom_number = count_heavy_atoms(pattern) - count_heavy_atoms(target)
    if difference_of_heavy_atom_number != 0:
        print("Not microstates of the same molecule.")
        return False
    
    # Compare total charge
    total_charge_difference = calculate_total_charge_difference(pattern, target)
    if abs(total_charge_difference) != 1:
        print("Not a physical microstate pair. Total charge difference is ", total_charge_difference)
        return False
    
    # Find MCSS of two molecules and find equivalent heavy atoms
    label_dictionary = {}
    m, label_dictionary, mcss = get_mcss(pattern, target)
    #print("Length of label_dictionary: ", len(label_dictionary) )
    
    # how many heavy atoms have different protonation state?
    prot_deprot_heavy_atom_count = check_how_many_heavy_atoms_changed_protonation_state(pattern, target, label_dictionary)
    if prot_deprot_heavy_atom_count == 1 :
        print("These two microstates create a physical microscopic pKa pair.")
        return True
    else:
        print("Not a physical microstate pair.")
        return False
    
def are_different_microstates(pattern, target):
    
    # Compare number of heavy atoms
    difference_of_heavy_atom_number = count_heavy_atoms(pattern) - count_heavy_atoms(target)
    if difference_of_heavy_atom_number != 0:
        print("Not microstates of the same molecule.")
        return True
    
    # Compare total charge
    total_charge_difference = calculate_total_charge_difference(pattern, target)
    if abs(total_charge_difference) != 0:
        print("Can't be resonance structure or geometric isomer. Total charge difference is: ", total_charge_difference)
        return True
    
    # Compare H-counts on equivalent heavy atoms
    
    # Find MCSS of two molecules and find equivalent heavy atoms
    label_dictionary = {}
    m, label_dictionary, mcss = get_mcss(pattern, target)
    #print("Length of label_dictionary: ", len(label_dictionary) )
    
    # how many heavy atoms have different protonation state?
    prot_deprot_heavy_atom_count = check_how_many_heavy_atoms_changed_protonation_state(pattern, target, label_dictionary)
    if prot_deprot_heavy_atom_count == 0 :
        print("\nWarning! These two structures are related to the same microstate!")
        return False
    else:
        print("These two structures are unique microstates.")
        return True


## Example microstates to test the method

In [22]:
# a physical microscopic pKa pair, not resonance structure
SM24_micro011 = "COc1ccc(cc1)c2c3c(=[NH+]CCO)[nH]cnc3oc2c4ccc(cc4)OC" # HA+
SM24_micro013 = "COc1ccc(cc1)c2c3c([nH+]cnc3oc2c4ccc(cc4)OC)NCC[O-]" # A 

pattern = get_labeled_mol(SM24_micro011)
target = get_labeled_mol(SM24_micro013)

are_different_microstates(pattern, target)

Can't be resonance structure or geometric isomer. Total charge difference is:  1


True

In [23]:
# resonance pair
SM11_micro018 = "c1ccc(cc1)n2c3c(cn2)c([nH+]c[nH+]3)N"
SM11_micro020 = "c1ccc(cc1)n2c3c(cn2)c([nH+]c[nH+]3)N"

pattern = get_labeled_mol(SM11_micro018)
target = get_labeled_mol(SM11_micro020)

are_different_microstates(pattern, target)

Match 1:
pattern atoms: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 
target atoms:  0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 

pattern heavy atom labels: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 
target heavy atom labels:  1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 

match smiles = c1ccc(cc1)n2c3c(cn2)c([nH+]c[nH+]3)N
Exactly one match.
Number of heavy atoms with different number of hydrogens:  0

Warning! These two structures are related to the same microstate!


False

In [24]:
# resonance pair
SM03_micro005 = "c1ccc(cc1)Cc2n[nH+]c(s2)[N-]C(=C3C=CC=[S+]3)[O-]"
SM05_micro008 = "c1ccc(cc1)Cc2[n-][nH+]c([s+]2)[N-]C(=O)c3cccs3"

pattern = get_labeled_mol(SM03_micro005)
target = get_labeled_mol(SM05_micro008)

are_different_microstates(pattern, target)

Match 1:
pattern atoms: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
target atoms:  0 1 2 3 4 5 6 7 8 9 10 11 12 13 15 16 17 18 19 14 

pattern heavy atom labels: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
target heavy atom labels:  1 2 3 4 5 6 7 8 9 10 11 12 13 14 16 17 18 19 20 15 

match smiles = c1ccc(cc1)Cc2[n-][nH+]c([s+]2)[N-]C(=O)c3cccs3
Exactly one match.
Number of heavy atoms with different number of hydrogens:  0

Warning! These two structures are related to the same microstate!


False

In [29]:
# not physical pairs, not resonance.

SM11_micro018 = "c1ccc(cc1)n2c3c(cn2)c([nH+]c[nH+]3)N"
SM11_micro031 = "c1ccc(cc1)n2c3c(c[nH+]2)c([nH+]cn3)N"

pattern = get_labeled_mol(SM11_micro018)
target = get_labeled_mol(SM11_micro031)

are_different_microstates(pattern, target)

Match 1:
pattern atoms: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 
target atoms:  0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 

pattern heavy atom labels: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 
target heavy atom labels:  1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 

match smiles = c1ccc(cc1)n2c3c(c[nH+]2)c([nH+]cn3)N
Exactly one match.
Number of heavy atoms with different number of hydrogens:  2
These two structures are unique microstates.


True

In [26]:
# not physical pairs, not resonance.

SM11_micro018 = "c1ccc(cc1)n2c3c(cn2)c([nH+]c[nH+]3)N"
SM11_micro017 = "c1ccc(cc1)n2c3c(cn2)c(nc[nH+]3)[NH3+]"

pattern = get_labeled_mol(SM11_micro017)
target = get_labeled_mol(SM11_micro018)

are_different_microstates(pattern, target)

Match 1:
pattern atoms: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 
target atoms:  0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 

pattern heavy atom labels: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 
target heavy atom labels:  1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 

match smiles = c1ccc(cc1)n2c3c(cn2)c([nH+]c[nH+]3)N
Exactly one match.
Number of heavy atoms with different number of hydrogens:  2
These two structures are unique microstates.


True

## Detect replicate microstates in all molecules
All small molecules except SM18 is scanned. SM18 has 74 microstates so I need a quicker method for that.

In [36]:
path_to_input_microstates = "corrections_for_v1_3_2/"
molecule_IDs_with_replicate_microstates = []

for m in range(24):
    mol_ID = "SM"+str(m+1).zfill(2)
    
    # Skip SM11
    #if (mol_ID == "SM11") :
    #    continue
        
    print("--------------------------------------------")
    print(mol_ID, "...")
    
    input_file_name = path_to_input_microstates + mol_ID +"_correction.csv"
    df_microstates = pd.read_csv(input_file_name)
    
    
    # Check if there is any deprecated microstate and create corrected microstates list accordingly.

    correction = df_microstates["correction"]
    deprecated_boolean = correction.isin(["deprecated"])

    deprecated_label = False
    for b in deprecated_boolean:
        if b == False:
            continue
        if b == True:
            print("Deprecated microstate found.")
            deprecated_label = True

    if(deprecated_label):
        df_corrected = df_microstates.loc[df_microstates["correction"] != "deprecated"]
    else:
        df_corrected = df_microstates
        
    # Check for resonance structure pairs in corrected list
    
    resonance_pairs_list = []

    # Pairwise comparison of microstates
    for i, row in enumerate(df_corrected.iterrows()):
        smiles1 = row[1][1]
        microstate_ID1 = row[1][0]
        # print(microstate_ID1, ":", smiles1)

        for j, row in enumerate(df_corrected.iterrows()):
            if j<= i:
                continue
            else:
                smiles2 = row[1][1]
                microstate_ID2 = row[1][0]

                print("Comparing {} and {}...".format(microstate_ID1, microstate_ID2))
                
                # create OEmol objects for 2 molecules
                pattern = get_labeled_mol(smiles1)
                target = get_labeled_mol(smiles2)
                
                #if total charge is the same test resonance
                if(compare_total_charge(pattern, target)):
                    print("Total charge is the same. Must compare if these are replicate structures...")
                    print(microstate_ID1, microstate_ID2)
                
                    if not are_different_microstates(pattern, target): # if replicate structures
                        # print("{} and {} are related the same microstate.".format(microstate_ID1, microstate_ID2))
                        resonance_pairs_list.append((microstate_ID1, microstate_ID2))
                else:
                    continue
    
    print(mol_ID, "resonance structures:")
    for pair in resonance_pairs_list:
        print(pair)
    print("\n")
                  
    if len(resonance_pairs_list) > 0:
        molecule_IDs_with_replicate_microstates.append(mol_ID)

--------------------------------------------
SM01 ...
Deprecated microstate found.
Deprecated microstate found.
Comparing SM01_micro001 and SM01_micro002...
Comparing SM01_micro001 and SM01_micro004...
Comparing SM01_micro001 and SM01_micro005...
Comparing SM01_micro001 and SM01_micro006...
Comparing SM01_micro001 and SM01_micro008...
Comparing SM01_micro001 and SM01_micro009...
Total charge is the same. Must compare if these are replicate structures...
SM01_micro001 SM01_micro009
Match 1:
pattern atoms: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 
target atoms:  0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 

pattern heavy atom labels: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 
target heavy atom labels:  1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 

match smiles = c1cc2c(cc1O)c3c(o2)C(=O)[NH2+]CCC3
Exactly one match.
Number of heavy atoms with different number of hydrogens:  2
These two structures are unique microstates.
Comparing SM01_micro001 and SM01_micro010...
Comparing SM01_micro002 and SM01_micro004

pattern atoms: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
target atoms:  15 16 17 12 13 14 8 9 10 11 7 6 18 19 20 4 5 1 0 2 3 

pattern heavy atom labels: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 
target heavy atom labels:  16 17 18 13 14 15 9 10 11 12 8 7 19 20 21 5 6 2 1 3 4 

match smiles = c1ccc2c(c1)c([nH+]cn2)[NH2+]c3cccc(c3)C(F)(F)F
Exactly one match.
Number of heavy atoms with different number of hydrogens:  2
These two structures are unique microstates.
Comparing SM02_micro008 and SM02_micro014...
Comparing SM02_micro011 and SM02_micro012...
Comparing SM02_micro011 and SM02_micro013...
Comparing SM02_micro011 and SM02_micro014...
Comparing SM02_micro012 and SM02_micro013...
Total charge is the same. Must compare if these are replicate structures...
SM02_micro012 SM02_micro013
Match 1:
pattern atoms: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
target atoms:  0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 

pattern heavy atom labels: 1 2 3

target heavy atom labels:  1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 

match smiles = c1ccc(cc1)Cc2nnc(s2)NC(=O)c3cccs3
Exactly one match.
Number of heavy atoms with different number of hydrogens:  2
These two structures are unique microstates.
Comparing SM03_micro009 and SM03_micro013...
Total charge is the same. Must compare if these are replicate structures...
SM03_micro009 SM03_micro013
Match 1:
pattern atoms: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
target atoms:  0 1 2 3 4 5 6 7 8 9 10 19 11 12 13 14 15 16 17 18 

pattern heavy atom labels: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
target heavy atom labels:  1 2 3 4 5 6 7 8 9 10 11 20 12 13 14 15 16 17 18 19 

match smiles = c1ccc(cc1)Cc2n[nH]c(=NC(=O)c3cccs3)s2
Exactly one match.
Number of heavy atoms with different number of hydrogens:  2
These two structures are unique microstates.
Comparing SM03_micro009 and SM03_micro014...
Comparing SM03_micro009 and SM03_micro015...
Comparing SM03_micro009 and 

match smiles = c1ccc(cc1)CC2=N[NH2+]/C(=N\C(=O)c3cccs3)/S2
Exactly one match.
Number of heavy atoms with different number of hydrogens:  2
These two structures are unique microstates.
Comparing SM03_micro016 and SM03_micro021...
Comparing SM03_micro016 and SM03_micro022...
Total charge is the same. Must compare if these are replicate structures...
SM03_micro016 SM03_micro022
Match 1:
pattern atoms: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
target atoms:  0 1 2 3 4 5 6 7 8 9 10 19 11 12 13 14 15 16 17 18 

pattern heavy atom labels: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 
target heavy atom labels:  1 2 3 4 5 6 7 8 9 10 11 20 12 13 14 15 16 17 18 19 

match smiles = c1ccc(cc1)Cc2n[nH]/c(=N\C(=[OH+])c3cccs3)/s2
Exactly one match.
Number of heavy atoms with different number of hydrogens:  2
These two structures are unique microstates.
Comparing SM03_micro016 and SM03_micro023...
Total charge is the same. Must compare if these are replicate structures...
SM03_micro016 S

pattern atoms: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 
target atoms:  0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 

pattern heavy atom labels: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
target heavy atom labels:  1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 

match smiles = c1ccc2c(c1)c([nH+]c[nH+]2)NCc3ccc(cc3)Cl
Exactly one match.
Number of heavy atoms with different number of hydrogens:  2
These two structures are unique microstates.
Comparing SM04_micro013 and SM04_micro015...
Total charge is the same. Must compare if these are replicate structures...
SM04_micro013 SM04_micro015
Match 1:
pattern atoms: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 
target atoms:  0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 

pattern heavy atom labels: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
target heavy atom labels:  1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 

match smiles = c1ccc2c(c1)c([nH+]cn2)[NH2+]Cc3ccc(cc3)Cl
Exactly one match.
Number of heavy atoms wit

...

Match 1:
pattern atoms: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 
target atoms:  0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 

pattern heavy atom labels: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 
target heavy atom labels:  1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

match smiles = COc1ccc(cc1)c2c3c(ncnc3oc2c4ccc(cc4)OC)[N-]CC[OH2+]
Exactly one match.
Number of heavy atoms with different number of hydrogens:  2
These two structures are unique microstates.
Comparing SM24_micro009 and SM24_micro025...
Comparing SM24_micro009 and SM24_micro027...
Comparing SM24_micro009 and SM24_micro028...
Total charge is the same. Must compare if these are replicate structures...
SM24_micro009 SM24_micro028
Match 1:
pattern atoms: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 
target atoms:  0 1 2 3 4 5 6 7 8 9 14 13 12 11 10 19 20 21

Match 1:
pattern atoms: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 
target atoms:  0 1 2 3 4 5 27 28 6 18 19 20 21 22 23 24 25 26 17 16 7 8 9 10 11 14 15 12 13 

pattern heavy atom labels: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 
target heavy atom labels:  1 2 3 4 5 6 28 29 7 19 20 21 22 23 24 25 26 27 18 17 8 9 10 11 12 15 16 13 14 

match smiles = COc1ccc(cc1)c2c3c([nH+]cnc3[NH2+]CC[O-])oc2c4ccc(cc4)OC
Exactly one match.
Number of heavy atoms with different number of hydrogens:  4
These two structures are unique microstates.
Comparing SM24_micro011 and SM24_micro030...
Total charge is the same. Must compare if these are replicate structures...
SM24_micro011 SM24_micro030
Match 1:
pattern atoms: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 
target atoms:  0 1 2 3 4 5 27 28 6 18 19 20 21 22 23 24 25 26 17 16 7 8 9 10 11 14 15 12 13 

pattern heavy atom labels: 1 2 3 4 5 6 7 8 9 10 11

Match 1:
pattern atoms: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 
target atoms:  0 1 2 3 4 5 27 28 6 18 19 24 25 26 17 16 7 8 9 10 11 14 15 12 13 20 21 22 23 

pattern heavy atom labels: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 
target heavy atom labels:  1 2 3 4 5 6 28 29 7 19 20 25 26 27 18 17 8 9 10 11 12 15 16 13 14 21 22 23 24 

match smiles = COc1ccc(cc1)c2c3c([nH+]cnc3[NH2+]CC[O-])oc2c4ccc(cc4)OC
Exactly one match.
Number of heavy atoms with different number of hydrogens:  2
These two structures are unique microstates.
Comparing SM24_micro017 and SM24_micro030...
Total charge is the same. Must compare if these are replicate structures...
SM24_micro017 SM24_micro030
Match 1:
pattern atoms: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 
target atoms:  0 1 2 3 4 5 27 28 6 18 19 24 25 26 17 16 7 8 9 10 11 14 15 12 13 20 21 22 23 

pattern heavy atom labels: 1 2 3 4 5 6 7 8 9 10 11

Match 1:
pattern atoms: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 
target atoms:  0 1 2 3 4 5 27 28 6 18 19 24 25 26 17 16 7 8 9 10 11 14 15 12 13 20 21 22 23 

pattern heavy atom labels: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 
target heavy atom labels:  1 2 3 4 5 6 28 29 7 19 20 25 26 27 18 17 8 9 10 11 12 15 16 13 14 21 22 23 24 

match smiles = COc1ccc(cc1)c2c3c([nH+]cnc3oc2c4ccc(cc4)OC)[NH2+]CCO
Exactly one match.
Number of heavy atoms with different number of hydrogens:  2
These two structures are unique microstates.
Comparing SM24_micro022 and SM24_micro034...
Total charge is the same. Must compare if these are replicate structures...
SM24_micro022 SM24_micro034
Match 1:
pattern atoms: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 
target atoms:  0 1 2 3 4 5 27 28 6 18 19 24 25 26 17 16 7 8 9 10 11 14 15 12 13 20 21 22 23 

pattern heavy atom labels: 1 2 3 4 5 6 7 8 9 10 11 12

Match 1:
pattern atoms: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 
target atoms:  0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 

pattern heavy atom labels: 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 
target heavy atom labels:  1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

match smiles = COc1ccc(cc1)c2c3c([nH+]c[nH+]c3oc2c4ccc(cc4)OC)NCC[O-]
Exactly one match.
Number of heavy atoms with different number of hydrogens:  2
These two structures are unique microstates.
Comparing SM24_micro030 and SM24_micro032...
Comparing SM24_micro030 and SM24_micro033...
Comparing SM24_micro030 and SM24_micro034...
Comparing SM24_micro030 and SM24_micro035...
Comparing SM24_micro031 and SM24_micro032...
Comparing SM24_micro031 and SM24_micro033...
Comparing SM24_micro031 and SM24_micro034...
Comparing SM24_micro031 and SM24_micro035...
Comparing SM24_micro032 and SM24_mic

In [37]:
molecule_IDs_with_replicate_microstates

['SM10', 'SM18', 'SM21', 'SM23', 'SM24']

## Microstate pairs to check manually

### SM10
('SM10_micro015', 'SM10_micro029')

### SM18
('SM18_micro001', 'SM18_micro041')
('SM18_micro005', 'SM18_micro008')
('SM18_micro009', 'SM18_micro035')
('SM18_micro010', 'SM18_micro073')
('SM18_micro011', 'SM18_micro039')
('SM18_micro013', 'SM18_micro067')
('SM18_micro015', 'SM18_micro061')
('SM18_micro019', 'SM18_micro029')
('SM18_micro025', 'SM18_micro027')
('SM18_micro032', 'SM18_micro040')
('SM18_micro037', 'SM18_micro044')
('SM18_micro038', 'SM18_micro066')
('SM18_micro043', 'SM18_micro053')
('SM18_micro046', 'SM18_micro063')

### SM21
('SM21_micro016', 'SM21_micro025')

### SM23
('SM23_micro001', 'SM23_micro008')
('SM23_micro001', 'SM23_micro020')
('SM23_micro004', 'SM23_micro011')
('SM23_micro005', 'SM23_micro016')
('SM23_micro006', 'SM23_micro014')
('SM23_micro006', 'SM23_micro028')
('SM23_micro008', 'SM23_micro020')
('SM23_micro014', 'SM23_micro028')
('SM23_micro015', 'SM23_micro027')
('SM23_micro015', 'SM23_micro029')
('SM23_micro015', 'SM23_micro036')
('SM23_micro027', 'SM23_micro029')
('SM23_micro027', 'SM23_micro036')
('SM23_micro029', 'SM23_micro036')


### SM24
('SM24_micro007', 'SM24_micro008') - correct! Deprecate: SM24_micro008
